In [54]:
import sys,os,time
import pandas as pd
import sqlite3
day = time.strftime("%d_%m_%y")
tcrdDB = 'tcrd_v5.2.0.db'
targetDB = '/ssddata/sdecesco/data/Projects/Database_druggability/databases_sqlite/TargetDB_06_08_18.db'
connector = sqlite3.connect(tcrdDB)
tdb = sqlite3.connect(targetDB)

In [2]:
tcrd_id = """SELECT id as tcrd_id,uniprot as Target_id FROM protein"""
df_id = pd.read_sql(tcrd_id,con=connector)
df_id.to_pickle('tcrd_id_'+day+'.pickle')

In [19]:
tcrd_target = """SELECT id as tcrd_id, tdl as Pharos_class, fam as protein_family, famext as protein_family_detail FROM target"""
df_target = pd.read_sql(tcrd_target,con=connector)
df_target.to_pickle('tcrd_target_'+day+'.pickle')

In [33]:
tcrd_info = """SELECT * FROM tdl_info """
df_info = pd.read_sql(tcrd_info,con=connector)
col_to_keep = ['Ab Count', 'MAb Count', 'NCBI Gene PubMed Count', 'EBI Total Patent Count', 'PubTator Score',
               'JensenLab PubMed Score']
df_info = df_info[df_info['itype'].isin(col_to_keep)]
df_info.loc[:,'value'] = df_info['number_value'].fillna(0) + df_info['integer_value'].fillna(0)
df_info = df_info.pivot(index='protein_id', columns='itype', values='value')
for col in col_to_keep:
    if col not in df_info.columns:
        df_info[col] = 0

df_info.index = df_info.index.astype(int)
df_info.reset_index(inplace=True)
df_info.to_pickle('tcrd_info_'+day+'.pickle')


In [34]:
tcrd_patent = """SELECT * FROM patent_count"""
df_patent = pd.read_sql(tcrd_patent,con=connector)
df_patent.drop(columns=['id'],inplace=True)
df_patent.to_pickle('tcrd_patent_'+day+'.pickle')

In [40]:
tcrd_disease = """SELECT TI.protein_id,TI.disease_id,T.doid,TI.score,T.name,DP.parent
FROM
tinx_importance TI
LEFT JOIN tinx_disease T ON TI.disease_id = T.id
LEFT JOIN do_parent DP on T.doid = DP.doid
ORDER BY TI.score DESC"""
df_disease = pd.read_sql(tcrd_disease,con=connector)
df_disease = df_disease[df_disease.score>1]
df_disease.to_pickle('tcrd_disease_'+day+'.pickle')

In [45]:
tcrd_novelty = """SELECT score,protein_id FROM tinx_novelty"""
df_novelty = pd.read_sql(tcrd_novelty,con=connector)
df_novelty.to_pickle('tcrd_novelty_'+day+'.pickle')

In [55]:
df_info.to_sql('tcrd_info',con=tdb,if_exists='append',index=False)

/ssddata/sdecesco/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:1534: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [59]:
df_novelty

,score,protein_id
0,0.052667,6710
1,0.017907,17415
2,0.003140,203
3,0.012154,3360
4,0.954545,18898
5,0.019259,11058
6,0.109461,13500
7,0.436975,16283
8,0.014412,3748
9,0.169899,1080
